## Model Interpreter Example Notebook for a Non-Standard Model

### Setup

Will install a package with a rarer model type. Completed separately from the development as this is only used as an example and is not used in any other package file

In [ ]:
pip install imbalanced-learn

In [2]:
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

import sys

sys.path.append("../")

import model_interpreter
from model_interpreter.interpreter import ModelInterpreter

In [3]:
model_interpreter.__version__

'1.0.0'

### Prepare Data

The non-standard model is the BaggingClassifier

In [4]:
X, y = make_classification(
    n_samples=1000,
    n_features=4,
    n_informative=2,
    n_redundant=0,
    random_state=0,
    shuffle=False,
)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42
)

In [6]:
feature_names = ["feature1", "feature2", "feature3", "feature4"]

### Train model

Instantiate the base classifier:

In [7]:
base_clf = DecisionTreeClassifier()

Instantiate BalancedBaggingClassifier

In [9]:
model = BalancedBaggingClassifier(
    estimator=base_clf,
    sampling_strategy="auto",
    replacement=False,
    random_state=42,
)

Fit the model:

In [10]:
model.fit(X_train, y_train)

BalancedBaggingClassifier(estimator=DecisionTreeClassifier(), random_state=42)

### Build single model contribution object

A model interpreter object is created by specifiying the feature names in the same order as used for building the model.

In [11]:
single_model_contribution = ModelInterpreter(feature_names)

### Define single row of data to get contributions for

In [12]:
single_row = X_test[0].reshape(1, -1)

### Build Explainer

The fit method needs to be called on the model to build the explainer. As the model is non-standard, the data also must be passed.

Additionally, n_samples defaults to 50 as the sample size to build the explainer from but can be increased/decreased

In [13]:
single_model_contribution.fit(model, X_train=X_train, is_classification=True)

Model is not tree-based, shap.TreeExplainer raised: Model type not yet supported by TreeExplainer: <class 'imblearn.ensemble._bagging.BalancedBaggingClassifier'>
Will try to fit linear explainer instead
Model is not linear, will try to fit kernel explainer with default sample size of 50


### Use the transform method to get feature contributions

The transform method is called on the single row to create the class output, which by default returns a sorted list of dictionaries of features with descending absolute contribution to the prediction in the format:

`[{feature_name: feature_contribution}, ... ]`

In [14]:
contribution_list = single_model_contribution.transform(
    single_row, return_feature_values=False
)

print(contribution_list)

  0%|          | 0/1 [00:00<?, ?it/s]

[{'Name': 'feature2', 'Value': -0.511}, {'Name': 'feature4', 'Value': 0.0086666667}, {'Name': 'feature3', 'Value': -0.0053333333}, {'Name': 'feature1', 'Value': 0.0036666667}]


return_feature_values can also be set to True so the return is:

`[{feature_name: (feature_value, feature_contribution)}, ... ]`

In [15]:
contribution_list = single_model_contribution.transform(
    single_row, return_feature_values=True, return_type="dicts"
)

print(contribution_list)

  0%|          | 0/1 [00:00<?, ?it/s]

[{'feature2': (-0.9405037129106437, -0.5109999999999999)}, {'feature4': (0.7635414664859205, 0.00866666666666649)}, {'feature3': (-0.48948241875503073, -0.0053333333333333635)}, {'feature1': (1.37105505746784, 0.003666666666666655)}]


this however is not implemented when return_type is the default value of "name_value_dicts"

In [16]:
contribution_list = single_model_contribution.transform(
    single_row, return_feature_values=True
)

print(contribution_list)

  0%|          | 0/1 [00:00<?, ?it/s]

[{'Name': 'feature2', 'Value': -0.511}, {'Name': 'feature4', 'Value': 0.0086666667}, {'Name': 'feature3', 'Value': -0.0053333333}, {'Name': 'feature1', 'Value': 0.0036666667}]
